In [ ]:
# Configs
%load_ext autoreload
%autoreload 3
import sys
sys.path.insert(0, "../")
from src_code.data_utils.dataset import GridDataset
from src_code.data_utils.dataset_utils import CellType
from src_code.data_utils.prompt_utils import prompt_generator
from src_code.eval_utils.eval import calculate_score, eval_results
from src_code.model_utils.claude import ClaudeModel
from io import BytesIO
import base64
import ast
from copy import deepcopy

model_name = "claude-3-5-haiku-20241022"
api_key = ""
model = ClaudeModel(model_name=model_name, api_key=api_key)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
message_tmp = {
    "role": "user",
    "content": None,
}
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

In [83]:
eval_num = 100
messages_ascii = []
messages_coor = []
for i in range(eval_num):
    _, grid_world = dataset[i]
    prompt_ascii = prompt_generator(grid_world, pure_language=False)
    prompt_coor = prompt_generator(grid_world, pure_language=True)
    message = deepcopy(message_tmp)
    message["content"] = prompt_ascii
    messages_ascii.append(message)
    message = deepcopy(message_tmp)
    message["content"] = prompt_coor
    messages_coor.append(message)


In [95]:
# model.batch_process(messages_ascii)
model.batch_process(messages_coor)

MessageBatch(id='msgbatch_0163qEGh2aPEG5QTaYWS7u2C', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 4, 4, 12, 3, 29, 758344, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2025, 4, 5, 12, 3, 29, 758344, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=100, succeeded=0), results_url=None, type='message_batch')
msgbatch_0163qEGh2aPEG5QTaYWS7u2C


In [96]:
model.batch_id

'msgbatch_0163qEGh2aPEG5QTaYWS7u2C'

In [100]:
model.get_batch_status()

Batch msgbatch_0163qEGh2aPEG5QTaYWS7u2C processing status is ended


In [101]:
# results_ascii = model.get_results()
results_coor = model.get_results()

Success! my-request-0
I'll solve this using a breadth-first search (BFS) algorithm to find the shortest path while avoiding obstacles.

Let me break down the problem:
- Start: (3, 3)
- Goal: (0, 2)
- Obstacles: [(1, 4), (2, 0), (2, 1), (3, 2), (3, 4), (4, 0)]
- Grid size: 5x5

After running the pathfinding algorithm, here's the shortest path:

('go up', 'go up', 'go left', 'go left', 'go up')

This sequence of moves takes the path from (3, 3) to (0, 2) avoiding all obstacles while finding the shortest route.

Let me verify the path:
1. Start at (3, 3)
2. go up -> (2, 3)
3. go up -> (1, 3)
4. go left -> (1, 2)
5. go left -> (1, 1)
6. go up -> (0, 2) - Goal reached!

The path successfully navigates to the goal cell using the minimum number of steps and avoiding all obstacles.
Success! my-request-1
I'll solve this path-finding problem using a breadth-first search (BFS) algorithm to find the shortest path while avoiding obstacles. Let me work through this systematically:

Start: (1, 4)
Goa

In [102]:
len(results_ascii)
len(results_coor)

100

In [103]:
import json
with open("../claude_results_ascii_raw.json", "w") as f:
    json.dump(results_ascii, f, indent=4)

with open("../claude_results_coor_raw.json", "w") as f:
    json.dump(results_coor, f, indent=4)


In [126]:
results_ascii = []
results_coor = []
with open('../claude_results_ascii_raw.json', 'r') as f:
    results_ascii = json.load(f)

with open('../claude_results_coor_raw.json', 'r') as f:
    results_coor = json.load(f)

In [1]:
%load_ext autoreload
%autoreload 3
import sys
sys.path.insert(0, "../")
import json
from src_code.data_utils.dataset import GridDataset
from src_code.eval_utils.eval import calculate_score, eval_results
import ast
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

In [3]:
path_results_ascii = []
path_results_coor = []

with open('../claude_results_coor_raw.json', 'r') as f:
    results_coor = json.load(f)

with open('../claude_results_ascii_raw.json', 'r') as f:
    results_ascii = json.load(f)

for i, result in enumerate(results_ascii):
    path = "('" + result.split("('")[1].split("\n\n")[0]
    # print(f"{i=}" + f"{ path=}")
    r = ast.literal_eval(path)
    if type(r) == str:
        r = (r,)
    path_results_ascii.append(r)
    # print(f"{i=}" + f"path={r}")

for i, result in enumerate(results_coor):
    path = "('" + result.split("('")[1].split("\n\n")[0]
    # print(f"{i=}" + f"{ path=}")
    r = ast.literal_eval(path)
    if type(r) == str:
        r = (r,)
    path_results_coor.append(r)
    # print(f"{i=}" + f"path={r}")

eval_result_ascii = eval_results(path_results_ascii, dataset)
eval_result_coor = eval_results(path_results_coor, dataset)
print(f"{eval_result_ascii=}")
print(f"{eval_result_coor=}")
result = {"ascii": eval_result_ascii, "coor": eval_result_coor}

# with open('../result_claude_100.json', 'w') as f:
#     json.dump(result, f, indent=4)

eval_result_ascii={'success rate (%)': 7.000000000000001, 'average collision': 0.94, 'average goal_distance': 2.78, 'average path_length_difference': 2.0, 'pass_through_goal': 29}
eval_result_coor={'success rate (%)': 31.0, 'average collision': 0.63, 'average goal_distance': 1.87, 'average path_length_difference': 1.45, 'pass_through_goal': 14}
